<a href="https://colab.research.google.com/github/masies/CRA/blob/main/evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from os import walk
import os
import regex

from google.colab import auth
auth.authenticate_user()
project_id = 'helical-loop-303918'
bucket_name = 'code_review_automation'
!gcloud config set project {project_id}


Updated property [core/project].


In [9]:
!rm -rf ./validation_eval/
!gsutil cp -r gs://code_review_automation/fine_tuning/HP_tuning/comment_v1/both/non_pretrained/slanted/validation_eval .

Copying gs://code_review_automation/fine_tuning/HP_tuning/comment_v1/both/non_pretrained/slanted/validation_eval/codeANDcomment_code_100000_predictions...
Copying gs://code_review_automation/fine_tuning/HP_tuning/comment_v1/both/non_pretrained/slanted/validation_eval/codeANDcomment_code_10200_predictions...
Copying gs://code_review_automation/fine_tuning/HP_tuning/comment_v1/both/non_pretrained/slanted/validation_eval/codeANDcomment_code_15000_predictions...
Copying gs://code_review_automation/fine_tuning/HP_tuning/comment_v1/both/non_pretrained/slanted/validation_eval/codeANDcomment_code_15300_predictions...
/ [4 files][  1.7 MiB/  1.7 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://code_review_automation/fine_tuning/HP

In [10]:
def eval(val_folder, tasks):
  for task in tasks:
    print(task)
    _, _, filenames = next(walk(val_folder))
    prediction_paths = [os.path.join(val_folder, f) for f in filenames if f.startswith(task) and f.endswith("predictions")]
    target_path = [os.path.join(val_folder, f) for f in filenames if f.startswith(task) and f.endswith("targets")][0]

    with open(target_path) as f:
        targets = [item.strip() for item in f.readlines()]

    for path in sorted(prediction_paths):
      step = regex.findall( task + "_(\d+)_predictions", path)[0]
      if step.endswith("000"):
        with open(path) as f:
          predictions = [item.strip() for item in f.readlines()]
        
        perfect_predictions = 0
        almost_perfect_predictions = 0
        for pred, target in zip(predictions, targets):
          if ' '.join(pred.strip().split()) == ' '.join(target.strip().split()):
            perfect_predictions += 1
          elif ''.join(pred.strip().split()) == ''.join(target.strip().split()):
            almost_perfect_predictions += 1
            index = next((i for i, (char1, char2) in enumerate(zip(pred, target)) if char1 != char2), None)
            
        print("perfect predictions at step " + step + " : ", round(perfect_predictions/len(predictions)*100,2))
        print("almost perfect predictions at step " + step + " : ", round((perfect_predictions + almost_perfect_predictions)/len(predictions)*100,2))
        print()

eval('validation_eval', ["code_code", "codeANDcomment_code"])
# eval('validation_eval', ["code_code"])
      


code_code
perfect predictions at step 100000 :  6.37
almost perfect predictions at step 100000 :  6.48

perfect predictions at step 15000 :  5.96
almost perfect predictions at step 15000 :  6.19

perfect predictions at step 20000 :  5.55
almost perfect predictions at step 20000 :  5.67

perfect predictions at step 25000 :  6.07
almost perfect predictions at step 25000 :  6.13

perfect predictions at step 30000 :  5.84
almost perfect predictions at step 30000 :  5.9

perfect predictions at step 35000 :  5.55
almost perfect predictions at step 35000 :  5.61

perfect predictions at step 40000 :  5.9
almost perfect predictions at step 40000 :  6.02

perfect predictions at step 45000 :  6.07
almost perfect predictions at step 45000 :  6.25

perfect predictions at step 50000 :  6.66
almost perfect predictions at step 50000 :  6.83

perfect predictions at step 55000 :  6.66
almost perfect predictions at step 55000 :  6.83

perfect predictions at step 60000 :  6.25
almost perfect predictions a

In [1]:
! pip install -q torch==1.4.0 -f https://download.pytorch.org/whl/cu101/torch_stable.html
! pip install -q transformers==3.5.0 fast-trees
! git clone -q https://github.com/microsoft/CodeXGLUE.git
!cd /content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU/ && python calc_code_bleu.py --refs ./../references.txt --hyp ./../predictions.txt --lang java --params 0.25,0.25,0.25,0.25

     |████████████████████████████████| 753.4MB 22kB/s 
ERROR: torchvision 0.9.0+cu101 has requirement torch==1.8.0, but you'll have torch 1.4.0 which is incompatible.
ERROR: torchtext 0.9.0 has requirement torch==1.8.0, but you'll have torch 1.4.0 which is incompatible.
     |████████████████████████████████| 1.3MB 4.3MB/s 
     |████████████████████████████████| 890kB 17.0MB/s 
     |████████████████████████████████| 1.1MB 25.5MB/s 
     |████████████████████████████████| 2.9MB 39.5MB/s 
     |████████████████████████████████| 163kB 40.1MB/s 
     |████████████████████████████████| 112kB 44.0MB/s 
     |████████████████████████████████| 71kB 5.8MB/s 


In [2]:
!cd /content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU/ && python calc_code_bleu.py --refs ./../../../../../tar --hyp ./../../../../../pred --lang java --params 0.25,0.25,0.25,0.25

Traceback (most recent call last):
  File "calc_code_bleu.py", line 29, in <module>
    for file in args.refs]
  File "calc_code_bleu.py", line 29, in <listcomp>
    for file in args.refs]
FileNotFoundError: [Errno 2] No such file or directory: './../../../../../tar'
